In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import seaborn as sns
# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline

In [2]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = '192.168.60.144'
    conn_info["sqlport"] = 6432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == '192.168.60.144') & (conn_info["sqlport"]=='6432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


In [3]:
from sqlalchemy import create_engine,text
con= create_engine('postgresql://eicu@192.168.60.144:6432/eicu')

In [4]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'
porcesseddir = '/home/mei/nas/docker/processedData/'

## lab

In [22]:
create_table_query =  query_schema +"""
CREATE TABLE pivoted_bg as
-- get blood gas measures
with vw0 as
(
  select
      patientunitstayid
    , labname
    , labresultoffset
    , labresultrevisedoffset
  from lab
  where labname in
  (
        'paO2'
      , 'paCO2'
      , 'pH'
      , 'FiO2'
      , 'anion gap'
      , 'Base Deficit'
      , 'Base Excess'
      , 'PEEP'
  )
  group by patientunitstayid, labname, labresultoffset, labresultrevisedoffset
  having count(distinct labresult)<=1
)
-- get the last lab to be revised
, vw1 as
(
  select
      lab.patientunitstayid
    , lab.labname
    , lab.labresultoffset
    , lab.labresultrevisedoffset
    , lab.labresult
    , ROW_NUMBER() OVER
        (
          PARTITION BY lab.patientunitstayid, lab.labname, lab.labresultoffset
          ORDER BY lab.labresultrevisedoffset DESC
        ) as rn
  from lab
  inner join vw0
    ON  lab.patientunitstayid = vw0.patientunitstayid
    AND lab.labname = vw0.labname
    AND lab.labresultoffset = vw0.labresultoffset
    AND lab.labresultrevisedoffset = vw0.labresultrevisedoffset
  WHERE
     (lab.labname = 'paO2' and lab.labresult >= 15 and lab.labresult <= 720)
  OR (lab.labname = 'paCO2' and lab.labresult >= 5 and lab.labresult <= 250)
  OR (lab.labname = 'pH' and lab.labresult >= 6.5 and lab.labresult <= 8.5)
  OR (lab.labname = 'FiO2' and lab.labresult >= 0.2 and lab.labresult <= 1.0)
  -- we will fix fio2 units later
  OR (lab.labname = 'FiO2' and lab.labresult >= 20 and lab.labresult <= 100)
  OR (lab.labname = 'anion gap' and lab.labresult >= 0 and lab.labresult <= 300)
  OR (lab.labname = 'Base Deficit' and lab.labresult >= -100 and lab.labresult <= 100)
  OR (lab.labname = 'Base Excess' and lab.labresult >= -100 and lab.labresult <= 100)
  OR (lab.labname = 'PEEP' and lab.labresult >= 0 and lab.labresult <= 60)
)
select
    patientunitstayid
  , labresultoffset as chartoffset
  -- the aggregate (max()) only ever applies to 1 value due to the where clause
  , MAX(case
        when labname != 'FiO2' then null
        when labresult >= 20 then labresult/100.0
      else labresult end) as fio2
  , MAX(case when labname = 'paO2' then labresult else null end) as pao2
  , MAX(case when labname = 'paCO2' then labresult else null end) as paco2
  , MAX(case when labname = 'pH' then labresult else null end) as pH
  , MAX(case when labname = 'anion gap' then labresult else null end) as aniongap
  , MAX(case when labname = 'Base Deficit' then labresult else null end) as basedeficit
  , MAX(case when labname = 'Base Excess' then labresult else null end) as baseexcess
  , MAX(case when labname = 'PEEP' then labresult else null end) as peep
from vw1
where rn = 1
group by patientunitstayid, labresultoffset
order by patientunitstayid, labresultoffset;
"""

In [ ]:
with con.begin() as connection:
    connection.execute(text(create_table_query))  # Use text() to wrap the raw SQL

In [24]:

select_query = "SELECT * FROM pivoted_bg;"
df_bg = pd.read_sql_query(select_query, con)

print(df_bg)

         patientunitstayid  chartoffset  fio2  pao2  paco2     ph  aniongap  \
0                   141168          516   NaN   NaN    NaN    NaN      15.0   
1                   141168         1133   NaN   NaN    NaN    NaN      20.0   
2                   141168         1805  0.28  41.0   46.0  7.140       NaN   
3                   141168         2010  1.00  42.0   44.0  7.140       NaN   
4                   141168         2026   NaN  68.0   31.0  7.160      25.0   
...                    ...          ...   ...   ...    ...    ...       ...   
1464007            3353254         2610   NaN   NaN    NaN    NaN       3.0   
1464008            3353254         4144   NaN   NaN    NaN    NaN       5.0   
1464009            3353254         4237   NaN  57.0   34.3  7.412       NaN   
1464010            3353254         5558   NaN   NaN    NaN    NaN       6.0   
1464011            3353263           -7   NaN   NaN    NaN    NaN       3.0   

         basedeficit  baseexcess  peep  
0         

In [26]:
# df_bg.to_csv(porcesseddir + 'pivoted_bg.csv',  index=True)
# print("Data exported successfully to 'pivoted_bg'.")

Data exported successfully to 'pivoted_bg'.


## nursecharting

In [37]:
gcs_query =  query_schema +"""
WITH nc AS
(
    SELECT
        patientunitstayid,
        nursingchartoffset AS chartoffset,
        MIN(CASE
            WHEN nursingchartcelltypevallabel = 'Glasgow coma score'
             AND nursingchartcelltypevalname = 'GCS Total'
             AND nursingchartvalue ~ '^[-]?[0-9]+[.]?[0-9]*$'
             AND nursingchartvalue NOT IN ('-', '.')
                THEN CAST(nursingchartvalue AS numeric)
            WHEN nursingchartcelltypevallabel = 'Score (Glasgow Coma Scale)'
             AND nursingchartcelltypevalname = 'Value'
             AND nursingchartvalue ~ '^[-]?[0-9]+[.]?[0-9]*$'
             AND nursingchartvalue NOT IN ('-', '.')
                THEN CAST(nursingchartvalue AS numeric)
            ELSE NULL END) AS gcs,
        MIN(CASE
            WHEN nursingchartcelltypevallabel = 'Glasgow coma score'
             AND nursingchartcelltypevalname = 'Motor'
             AND nursingchartvalue ~ '^[-]?[0-9]+[.]?[0-9]*$'
             AND nursingchartvalue NOT IN ('-', '.')
                THEN CAST(nursingchartvalue AS numeric)
            ELSE NULL END) AS gcsmotor,
        MIN(CASE
            WHEN nursingchartcelltypevallabel = 'Glasgow coma score'
             AND nursingchartcelltypevalname = 'Verbal'
             AND nursingchartvalue ~ '^[-]?[0-9]+[.]?[0-9]*$'
             AND nursingchartvalue NOT IN ('-', '.')
                THEN CAST(nursingchartvalue AS numeric)
            ELSE NULL END) AS gcsverbal,
        MIN(CASE
            WHEN nursingchartcelltypevallabel = 'Glasgow coma score'
             AND nursingchartcelltypevalname = 'Eyes'
             AND nursingchartvalue ~ '^[-]?[0-9]+[.]?[0-9]*$'
             AND nursingchartvalue NOT IN ('-', '.')
                THEN CAST(nursingchartvalue AS numeric)
            ELSE NULL END) AS gcseyes
    FROM nursecharting
    WHERE nursingchartcelltypecat IN ('Scores', 'Other Vital Signs and Infusions')
    GROUP BY patientunitstayid, nursingchartoffset
),
ncproc AS
(
    SELECT
        patientunitstayid,
        chartoffset,
        CASE WHEN gcs > 2 AND gcs < 16 THEN gcs ELSE NULL END AS gcs,
        gcsmotor, gcsverbal, gcseyes
    FROM nc
)
SELECT
    patientunitstayid,
    chartoffset,
    gcs,
    gcsmotor, gcsverbal, gcseyes
FROM ncproc
WHERE gcs IS NOT NULL
   OR gcsmotor IS NOT NULL
   OR gcsverbal IS NOT NULL
   OR gcseyes IS NOT NULL
ORDER BY patientunitstayid;

"""

In [38]:
df_gcs = pd.read_sql_query(gcs_query, con)


In [39]:
print(df_gcs)

         patientunitstayid  chartoffset   gcs  gcsmotor  gcsverbal  gcseyes
0                   141178         -277  12.0       NaN        NaN      NaN
1                   141179           12   9.0       NaN        NaN      NaN
2                   141179          216  12.0       NaN        NaN      NaN
3                   141179          522  12.0       NaN        NaN      NaN
4                   141179          702  12.0       NaN        NaN      NaN
...                    ...          ...   ...       ...        ...      ...
3451783            3353263         4306  15.0       6.0        5.0      4.0
3451784            3353263         4837  15.0       6.0        5.0      4.0
3451785            3353263         5651  15.0       6.0        5.0      4.0
3451786            3353263         7100  15.0       6.0        5.0      4.0
3451787            3353263         8525  15.0       6.0        5.0      4.0

[3451788 rows x 6 columns]


In [40]:
# df_gcs.to_csv(porcesseddir + 'pivoted_gcs.csv',  index=True)
# print("Data exported successfully to 'pivoted_gcs'.")

Data exported successfully to 'pivoted_gcs'.


## infusiondrug
- 从 infusiondrug 表中提取和处理多种药物（主要是一些用于支持循环系统的药物，如多巴胺、去甲肾上腺素、肾上腺素等）的输注记录，并将这些数据按患者ID和时间进行汇总处理，最终结果存储在一个名为 pivoted_infusion 的表中
- max(case ...) 判断药物是否存在
- 过滤掉那些没有任何药物使用的记录，最终输出包含至少一种药物使用的记录

In [7]:
create_table_infusion =  query_schema +"""
CREATE TABLE pivoted_infusion as
with vw0 as
(
  select
    patientunitstayid
    , infusionoffset
    -- TODO: need dopamine rate
    , max(case when drugname in
              (
                   'Dopamine'
                 , 'Dopamine ()'
                 , 'DOPamine MAX 800 mg Dextrose 5% 250 ml  Premix (mcg/kg/min)'
                 , 'Dopamine (mcg/hr)'
                 , 'Dopamine (mcg/kg/hr)'
                 , 'dopamine (mcg/kg/min)'
                 , 'Dopamine (mcg/kg/min)'
                 , 'Dopamine (mcg/min)'
                 , 'Dopamine (mg/hr)'
                 , 'Dopamine (ml/hr)'
                 , 'Dopamine (nanograms/kg/min)'
                 , 'DOPamine STD 15 mg Dextrose 5% 250 ml  Premix (mcg/kg/min)'
                 , 'DOPamine STD 400 mg Dextrose 5% 250 ml  Premix (mcg/kg/min)'
                 , 'DOPamine STD 400 mg Dextrose 5% 500 ml  Premix (mcg/kg/min)'
                 , 'Dopamine (Unknown)'
              )
              -- note: no rows found for inotropin
                then 1
              else null end
            ) as dopamine

    -- this like statement is pretty reliable - no false positives when I checked
    -- also catches the brand name dobutrex
    , max(case when lower(drugname) like '%dobu%' then 1 else null end) as dobutamine
    , max(case
              when drugname in
              (
                 'Norepinephrine'
               , 'Norepinephrine ()'
               , 'Norepinephrine MAX 32 mg Dextrose 5% 250 ml (mcg/min)'
               , 'Norepinephrine MAX 32 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine (mcg/hr)'
               , 'Norepinephrine (mcg/kg/hr)'
               , 'Norepinephrine (mcg/kg/min)'
               , 'Norepinephrine (mcg/min)'
               , 'Norepinephrine (mg/hr)'
               , 'Norepinephrine (mg/kg/min)'
               , 'Norepinephrine (mg/min)'
               , 'Norepinephrine (ml/hr)'
               , 'Norepinephrine STD 32 mg Dextrose 5% 282 ml (mcg/min)'
               , 'Norepinephrine STD 32 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine STD 4 mg Dextrose 5% 250 ml (mcg/min)'
               , 'Norepinephrine STD 4 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine STD 8 mg Dextrose 5% 250 ml (mcg/min)'
               , 'Norepinephrine STD 8 mg Dextrose 5% 500 ml (mcg/min)'
               , 'Norepinephrine (units/min)'
               , 'Norepinephrine (Unknown)'
               , 'norepinephrine Volume (ml)'
               , 'norepinephrine Volume (ml) (ml/hr)'
               -- levophed
              , 'Levophed (mcg/kg/min)'
              , 'levophed  (mcg/min)'
              , 'levophed (mcg/min)'
              , 'Levophed (mcg/min)'
              , 'Levophed (mg/hr)'
              , 'levophed (ml/hr)'
              , 'Levophed (ml/hr)'
              , 'NSS with LEVO (ml/hr)'
              , 'NSS w/ levo/vaso (ml/hr)'
              )
          then 1 else 0 end) as norepinephrine
    , max(case
          when drugname in
          (
             'Phenylephrine'
           , 'Phenylephrine ()'
           , 'Phenylephrine  MAX 100 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
           , 'Phenylephrine (mcg/hr)'
           , 'Phenylephrine (mcg/kg/min)'
           , 'Phenylephrine (mcg/kg/min) (mcg/kg/min)'
           , 'Phenylephrine (mcg/min)'
           , 'Phenylephrine (mcg/min) (mcg/min)'
           , 'Phenylephrine (mg/hr)'
           , 'Phenylephrine (mg/kg/min)'
           , 'Phenylephrine (ml/hr)'
           , 'Phenylephrine  STD 20 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
           , 'Phenylephrine  STD 20 mg Sodium Chloride 0.9% 500 ml (mcg/min)'
           , 'Volume (ml) Phenylephrine'
           , 'Volume (ml) Phenylephrine ()'
           -- neosynephrine is a synonym
           , 'neo-synephrine (mcg/min)'
           , 'neosynephrine (mcg/min)'
           , 'Neosynephrine (mcg/min)'
           , 'Neo Synephrine (mcg/min)'
           , 'Neo-Synephrine (mcg/min)'
           , 'NeoSynephrine (mcg/min)'
           , 'NEO-SYNEPHRINE (mcg/min)'
           , 'Neosynephrine (ml/hr)'
           , 'neosynsprine'
           , 'neosynsprine (mcg/kg/hr)'
          )
        then 1 else 0 end) as phenylephrine
    , max(case
            when drugname in
            (
                 'EPI (mcg/min)'
               , 'Epinepherine (mcg/min)'
               , 'Epinephrine'
               , 'Epinephrine ()'
               , 'EPINEPHrine(Adrenalin)MAX 30 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
               , 'EPINEPHrine(Adrenalin)STD 4 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
               , 'EPINEPHrine(Adrenalin)STD 4 mg Sodium Chloride 0.9% 500 ml (mcg/min)'
               , 'EPINEPHrine(Adrenalin)STD 7 mg Sodium Chloride 0.9% 250 ml (mcg/min)'
               , 'Epinephrine (mcg/hr)'
               , 'Epinephrine (mcg/kg/min)'
               , 'Epinephrine (mcg/min)'
               , 'Epinephrine (mg/hr)'
               , 'Epinephrine (mg/kg/min)'
               , 'Epinephrine (ml/hr)'
            ) then 1 else 0 end)
          as epinephrine
    , max(case
            when drugname in
            (
                'Vasopressin'
              , 'Vasopressin ()'
              , 'Vasopressin 20 Units Sodium Chloride 0.9% 100 ml (units/hr)'
              , 'Vasopressin 20 Units Sodium Chloride 0.9% 250 ml (units/hr)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (units/hr)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (units/kg/hr)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (units/min)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 100 ml (Unknown)'
              , 'Vasopressin 40 Units Sodium Chloride 0.9% 200 ml (units/min)'
              , 'Vasopressin (mcg/kg/min)'
              , 'Vasopressin (mcg/min)'
              , 'Vasopressin (mg/hr)'
              , 'Vasopressin (mg/min)'
              , 'vasopressin (ml/hr)'
              , 'Vasopressin (ml/hr)'
              , 'Vasopressin (units/hr)'
              , 'Vasopressin (units/kg/min)'
              , 'vasopressin (units/min)'
              , 'Vasopressin (units/min)'
              , 'VAsopressin (units/min)'
              , 'Vasopressin (Unknown)'
            ) then 1 else 0 end)
          as vasopressin
    , max(case when drugname in
      (
           'Milrinone'
         , 'Milrinone ()'
         , 'Milrinone (mcg/kg/hr)'
         , 'Milrinone (mcg/kg/min)'
         , 'Milrinone (ml/hr)'
         , 'Milrinone (Primacor) 40 mg Dextrose 5% 200 ml (mcg/kg/min)'
         , 'Milronone (mcg/kg/min)'
         , 'primacore (mcg/kg/min)'
      ) then 1 else 0 end)
      as milrinone
    , max(case when drugname in
      (
          'Hepain (ml/hr)'
        , 'Heparin'
        , 'Heparin ()'
        , 'Heparin 25,000 Unit/D5w 250 ml (ml/hr)'
        , 'Heparin 25000 Units Dextrose 5% 500 ml  Premix (units/hr)'
        , 'Heparin 25000 Units Dextrose 5% 500 ml  Premix (units/kg/hr)'
        , 'Heparin 25000 Units Dextrose 5% 950 ml  Premix (units/kg/hr)'
        , 'HEPARIN #2 (units/hr)'
        , 'Heparin 8000u/1L NS (ml/hr)'
        , 'Heparin-EKOS (units/hr)'
        , 'Heparin/Femoral Sheath   (units/hr)'
        , 'Heparin (mcg/kg/hr)'
        , 'Heparin (mcg/kg/min)'
        , 'Heparin (ml/hr)'
        , 'heparin (units/hr)'
        , 'Heparin (units/hr)'
        , 'HEPARIN (units/hr)'
        , 'Heparin (units/kg/hr)'
        , 'Heparin (Unknown)'
        , 'Heparin via sheath (units/hr)'
        , 'Left  Heparin (units/hr)'
        , 'NSS carrier heparin (ml/hr)'
        , 'S-Heparin (units/hr)'
        , 'Volume (ml) Heparin-heparin 25,000 units in 0.45 % sodium chloride 500 mL infusion'
        , 'Volume (ml) Heparin-heparin 25,000 units in 0.45 % sodium chloride 500 mL infusion (ml/hr)'
        , 'Volume (ml) Heparin-heparin 25,000 units in dextrose 500 mL infusion'
        , 'Volume (ml) Heparin-heparin 25,000 units in dextrose 500 mL infusion (ml/hr)'
        , 'Volume (ml) Heparin-heparin infusion 2 units/mL in 0.9% sodium chloride (ARTERIAL LINE)'
        , 'Volume (ml) Heparin-heparin infusion 2 units/mL in 0.9% sodium chloride (ARTERIAL LINE) (ml/hr)'
      ) then 1 else 0 end)
      as heparin
  from infusiondrug
  group by patientunitstayid, infusionoffset
)
select
  patientunitstayid
  , infusionoffset as chartoffset
  , dopamine::SMALLINT as dopamine
  , dobutamine::SMALLINT as dobutamine
  , norepinephrine::SMALLINT as norepinephrine
  , phenylephrine::SMALLINT as phenylephrine
  , epinephrine::SMALLINT as epinephrine
  , vasopressin::SMALLINT as vasopressin
  , milrinone::SMALLINT as milrinone
  , heparin::SMALLINT as heparin
from vw0
-- at least one of our drugs should be non-zero
where dopamine = 1
OR dobutamine = 1
OR norepinephrine = 1
OR phenylephrine = 1
OR epinephrine = 1
OR vasopressin = 1
OR milrinone = 1
OR heparin = 1
order by patientunitstayid, infusionoffset;
"""

In [8]:
with con.begin() as connection:
    connection.execute(text(create_table_infusion))  # Use text() to wrap the raw SQL

select_infusion = "SELECT * FROM pivoted_infusion;"
df_infusion = pd.read_sql_query(text(select_infusion),con)
print(df_infusion)


         patientunitstayid  chartoffset  dopamine  dobutamine  norepinephrine  \
0                   242040          457       NaN         NaN               0   
1                   242082           55       NaN         NaN               1   
2                   242082          125       NaN         NaN               1   
3                   242082          230       NaN         NaN               1   
4                   242082          275       NaN         NaN               1   
...                    ...          ...       ...         ...             ...   
1083069            3353251         4767       NaN         NaN               0   
1083070            3353251         4789       NaN         NaN               0   
1083071            3353251         4822       NaN         NaN               0   
1083072            3353263          328       NaN         NaN               0   
1083073            3353263          800       NaN         NaN               0   

         phenylephrine  epi

In [12]:
# df_infusion.to_csv(porcesseddir + 'pivoted_infusion.csv',  index=True)
# print("Data exported successfully to 'pivoted_infusion'.")

Data exported successfully to 'pivoted_infusion'.


## lab
- 从 lab 表中提取与实验室化验相关的数据，进行清理和去重，然后汇总每个患者在不同时间点的实验室结果

In [5]:
create_table_lab = query_schema + """
DROP TABLE IF EXISTS pivoted_lab CASCADE;
CREATE TABLE pivoted_lab as
-- remove duplicate labs if they exist at the same time
with vw0 as
(
  select
      patientunitstayid
    , labname
    , labresultoffset
    , labresultrevisedoffset
  from lab
  where labname in
  (
      'albumin'
    , 'total bilirubin'
    , 'BUN'
    , 'calcium'
    , 'chloride'
    , 'creatinine'
    , 'bedside glucose', 'glucose'
    , 'bicarbonate' -- HCO3
    , 'Total CO2'
    , 'Hct'
    , 'Hgb'
    , 'PT - INR'
    , 'PTT'
    , 'lactate'
    , 'platelets x 1000'
    , 'potassium'
    , 'sodium'
    , 'WBC x 1000'
    , '-bands'
    -- Liver enzymes
    , 'ALT (SGPT)'
    , 'AST (SGOT)'
    , 'alkaline phos.'
  )
  group by patientunitstayid, labname, labresultoffset, labresultrevisedoffset
  having count(distinct labresult)<=1
)
-- get the last lab to be revised
, vw1 as
(
  select
      lab.patientunitstayid
    , lab.labname
    , lab.labresultoffset
    , lab.labresultrevisedoffset
    , lab.labresult
    , ROW_NUMBER() OVER
        (
          PARTITION BY lab.patientunitstayid, lab.labname, lab.labresultoffset
          ORDER BY lab.labresultrevisedoffset DESC
        ) as rn
  from lab
  inner join vw0
    ON  lab.patientunitstayid = vw0.patientunitstayid
    AND lab.labname = vw0.labname
    AND lab.labresultoffset = vw0.labresultoffset
    AND lab.labresultrevisedoffset = vw0.labresultrevisedoffset
  -- only valid lab values
  WHERE
       (lab.labname = 'albumin' and lab.labresult >= 0.5 and lab.labresult <= 6.5)
    OR (lab.labname = 'total bilirubin' and lab.labresult >= 0.2 and lab.labresult <= 70.175)
    OR (lab.labname = 'BUN' and lab.labresult >= 1 and lab.labresult <= 280)
    OR (lab.labname = 'calcium' and lab.labresult > 0 and lab.labresult <= 9999)
    OR (lab.labname = 'chloride' and lab.labresult > 0 and lab.labresult <= 9999)
    OR (lab.labname = 'creatinine' and lab.labresult >= 0.1 and lab.labresult <= 28.28)
    OR (lab.labname in ('bedside glucose', 'glucose') and lab.labresult >= 25 and lab.labresult <= 1500)
    OR (lab.labname = 'bicarbonate' and lab.labresult >= 0 and lab.labresult <= 9999)
    OR (lab.labname = 'Total CO2' and lab.labresult >= 0 and lab.labresult <= 9999)
    -- will convert hct unit to fraction later
    OR (lab.labname = 'Hct' and lab.labresult >= 5 and lab.labresult <= 75)
    OR (lab.labname = 'Hgb' and lab.labresult >  0 and lab.labresult <= 9999)
    OR (lab.labname = 'PT - INR' and lab.labresult >= 0.5 and lab.labresult <= 15)
    OR (lab.labname = 'lactate' and lab.labresult >= 0.1 and lab.labresult <= 30)
    OR (lab.labname = 'platelets x 1000' and lab.labresult >  0 and lab.labresult <= 9999)
    OR (lab.labname = 'potassium' and lab.labresult >= 0.05 and lab.labresult <= 12)
    OR (lab.labname = 'PTT' and lab.labresult >  0 and lab.labresult <= 500)
    OR (lab.labname = 'sodium' and lab.labresult >= 90 and lab.labresult <= 215)
    OR (lab.labname = 'WBC x 1000' and lab.labresult > 0 and lab.labresult <= 100)
    OR (lab.labname = '-bands' and lab.labresult >= 0 and lab.labresult <= 100)
    OR (lab.labname = 'ALT (SGPT)' and lab.labresult > 0)
    OR (lab.labname = 'AST (SGOT)' and lab.labresult > 0)
    OR (lab.labname = 'alkaline phos.' and lab.labresult > 0)
)
select
    patientunitstayid
  , labresultoffset as chartoffset
  , MAX(case when labname = 'albumin' then labresult else null end) as albumin
  , MAX(case when labname = 'total bilirubin' then labresult else null end) as bilirubin
  , MAX(case when labname = 'BUN' then labresult else null end) as BUN
  , MAX(case when labname = 'calcium' then labresult else null end) as calcium
  , MAX(case when labname = 'chloride' then labresult else null end) as chloride
  , MAX(case when labname = 'creatinine' then labresult else null end) as creatinine
  , MAX(case when labname in ('bedside glucose', 'glucose') then labresult else null end) as glucose
  , MAX(case when labname = 'bicarbonate' then labresult else null end) as bicarbonate
  , MAX(case when labname = 'Total CO2' then labresult else null end) as TotalCO2
  , MAX(case when labname = 'Hct' then labresult else null end) as hematocrit
  , MAX(case when labname = 'Hgb' then labresult else null end) as hemoglobin
  , MAX(case when labname = 'PT - INR' then labresult else null end) as INR
  , MAX(case when labname = 'lactate' then labresult else null end) as lactate
  , MAX(case when labname = 'platelets x 1000' then labresult else null end) as platelets
  , MAX(case when labname = 'potassium' then labresult else null end) as potassium
  , MAX(case when labname = 'PTT' then labresult else null end) as ptt
  , MAX(case when labname = 'sodium' then labresult else null end) as sodium
  , MAX(case when labname = 'WBC x 1000' then labresult else null end) as wbc
  , MAX(case when labname = '-bands' then labresult else null end) as bands
  , MAX(case when labname = 'ALT (SGPT)' then labresult else null end) as alt
  , MAX(case when labname = 'AST (SGOT)' then labresult else null end) as ast
  , MAX(case when labname = 'alkaline phos.' then labresult else null end) as alp
from vw1
where rn = 1
group by patientunitstayid, labresultoffset
order by patientunitstayid, labresultoffset;

"""

In [6]:
with con.begin() as connection:
    connection.execute(text(create_table_lab))  # Use text() to wrap the raw SQL

select_lab = "SELECT * FROM pivoted_lab;"
df_lab = pd.read_sql_query(text(select_lab),con)
print(df_lab)

         patientunitstayid  chartoffset  albumin  bilirubin   bun  calcium  \
0                   141168          231      NaN        NaN   NaN      NaN   
1                   141168          516      3.1        2.6  26.0      8.8   
2                   141168         1133      3.3        4.1  27.0      9.2   
3                   141168         2026      3.0        5.2  29.0      8.5   
4                   141178         -280      4.0        0.4  11.0      8.0   
...                    ...          ...      ...        ...   ...      ...   
5314158            3353263         6446      NaN        NaN   NaN      NaN   
5314159            3353263         6919      NaN        NaN   NaN      NaN   
5314160            3353263         7303      NaN        NaN   NaN      NaN   
5314161            3353263         7490      NaN        NaN   NaN      NaN   
5314162            3353263         8926      NaN        NaN   NaN      NaN   

         chloride  creatinine  glucose  bicarbonate  ...  lacta

In [7]:
# df_lab.to_csv(porcesseddir + 'pivoted_lab.csv',  index=True)
# print("Data exported successfully to 'pivoted_lab'.")

Data exported successfully to 'pivoted_lab'.
